**Download DataSet**

In [1]:
!mkdir DataSet

In [2]:
cd DataSet

/content/DataSet


In [3]:
!gdown 16tiNjsITbWh-3J5laCgtBX0MaR6cVT2e

Downloading...
From: https://drive.google.com/uc?id=16tiNjsITbWh-3J5laCgtBX0MaR6cVT2e
To: /content/DataSet/ner_dataset.csv
100% 15.2M/15.2M [00:00<00:00, 43.6MB/s]


In [4]:
cd ..

/content


In [1]:
import matplotlib.pyplot as plt
from io import BytesIO
import pandas as pd
import seaborn as sn
import numpy as np
import requests
np.random.seed(0)
plt.style.use("ggplot")
import tensorflow as tf
print('Tensorflow version:', tf.__version__)
print('GPU detected:', tf.config.list_physical_devices('GPU'))

Tensorflow version: 2.16.1
GPU detected: []


In [2]:
data = pd.read_csv('ner_dataset.csv',encoding="latin1")
data = data.fillna(method="ffill")
data

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_9704\1162778166.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method="ffill")


,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
...,...,...,...,...
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O


Tạo một danh sách và sử dụng phương thức set() để lấy các giá trị trong cột “word” và loại bỏ giá trị trùng.\
Thêm phần đệm có tên “Endpad”.


In [3]:
words = list(set(data["Word"].values))
words.append("ENDPAD")
tags = list(set(data["Tag"].values))

n_words = len(words)
n_tags = len(tags)

print('Tổng số từ duy nhất:', n_words)
print('Số lượng nhãn thực thể:', n_tags)

Tổng số từ duy nhất: 35178
Số lượng nhãn thực thể: 17


In [4]:
class SentenceGetter(object):
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]

sentences = SentenceGetter(data).sentences

In [5]:
# padding

from tensorflow.keras.preprocessing.sequence import pad_sequences
max_len=50
tag2idx = {t: i for i, t in enumerate(tags)}
word2idx = {w: i for i, w in enumerate(words)}


X = [[word2idx[w[0]] for w in s] for s in sentences]
y = [[tag2idx[w[2]] for w in s] for s in sentences]

X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words-1)
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])

In [6]:
# train test split

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [8]:
# model
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense
from tensorflow.keras.layers import TimeDistributed, SpatialDropout1D, Bidirectional

input_word = Input(shape=(max_len,))
model = Embedding(input_dim=n_words, output_dim=50, input_length=max_len)(input_word)
model = SpatialDropout1D(0.1)(model)
model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(model)
out = TimeDistributed(Dense(n_tags, activation="softmax"))(model)
model = Model(input_word, out)
model.summary()

C:\Users\Lenovo\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 50, 50)         │     1,758,900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d               │ (None, 50, 50)         │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 50, 200)        │       120,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 50, 17)         │         3,417 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,883,117 (7.18 MB)

 Trainable params: 1,883,117 (7.18 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# train model

import time
start_time = time.time()

history = model.fit(x_train, y_train, validation_split= 0.1,
                    batch_size= 32, epochs=10, callbacks=[callback])

print("[{}] Completed!".format(time.time() - start_time))

Epoch 1/10
945/945 ━━━━━━━━━━━━━━━━━━━━ 51s 49ms/step - accuracy: 0.9358 - loss: 0.3427 - val_accuracy: 0.9807 - val_loss: 0.0675
Epoch 2/10
945/945 ━━━━━━━━━━━━━━━━━━━━ 54s 57ms/step - accuracy: 0.9829 - loss: 0.0590 - val_accuracy: 0.9848 - val_loss: 0.0498
Epoch 3/10
945/945 ━━━━━━━━━━━━━━━━━━━━ 48s 50ms/step - accuracy: 0.9882 - loss: 0.0390 - val_accuracy: 0.9857 - val_loss: 0.0462
Epoch 4/10
945/945 ━━━━━━━━━━━━━━━━━━━━ 47s 49ms/step - accuracy: 0.9905 - loss: 0.0310 - val_accuracy: 0.9860 - val_loss: 0.0457
Epoch 5/10
945/945 ━━━━━━━━━━━━━━━━━━━━ 49s 52ms/step - accuracy: 0.9915 - loss: 0.0271 - val_accuracy: 0.9860 - val_loss: 0.0473
Epoch 6/10
945/945 ━━━━━━━━━━━━━━━━━━━━ 49s 51ms/step - accuracy: 0.9924 - loss: 0.0236 - val_accuracy: 0.9859 - val_loss: 0.0481
Epoch 7/10
945/945 ━━━━━━━━━━━━━━━━━━━━ 49s 51ms/step - accuracy: 0.9932 - loss: 0.0209 - val_accuracy: 0.9860 - val_loss: 0.0488
Epoch 8/10
945/945 ━━━━━━━━━━━━━━━━━━━━ 47s 49ms/step - accuracy: 0.9938 - loss: 0.0189 - 

In [10]:
# evaluation
from seqeval.metrics import f1_score, classification_report

y_true = []
y_pred = []
for i in range(y_test.shape[0]):
  y_true.append(y_test[i])
  p = model.predict(np.array([x_test[i]]))
  p = np.argmax(p, axis=-1)
  y_pred.extend(p)

Y_true = []
Y_pred = []

for i, j in zip(range(len(y_true)), range(len(y_pred))):
  true = []
  pred = []
  for k in range(len(y_true[i])):
    true.append(tags[y_true[i][k]])
    pred.append(tags[y_pred[j][k]])
  Y_true.append(true)
  Y_pred.append(pred)

print('F1-score micro test set: {}%'.format(round(f1_score(Y_true, Y_pred, average='micro')*100,2)))
print('F1-score macro test set: {}%'.format(round(f1_score(Y_true, Y_pred, average='macro')*100,2)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 537ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━

In [11]:
# F1-score và Accuracy cho từng nhãn thực thể.
# F1-score
from collections.abc import Iterable
def flatten(lis):
     for item in lis:
         if isinstance(item, Iterable) and not isinstance(item, str):
             for x in flatten(item):
                 yield x
         else:
             yield item

YT = list(flatten(Y_true))
YP = list(flatten(Y_pred))


from sklearn.metrics import classification_report as sklearn_cs

print(sklearn_cs(YT, YP))

c:\Users\Lenovo\anaconda3\envs\preprocess\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Lenovo\anaconda3\envs\preprocess\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       B-art       0.00      0.00      0.00       140
       B-eve       1.00      0.02      0.03       113
       B-geo       0.85      0.89      0.87     11090
       B-gpe       0.95      0.94      0.95      4755
       B-nat       1.00      0.06      0.12        62
       B-org       0.81      0.66      0.73      5952
       B-per       0.85      0.78      0.82      5052
       B-tim       0.92      0.86      0.89      6118
       I-art       0.00      0.00      0.00       128
       I-eve       0.17      0.03      0.05       101
       I-geo       0.80      0.78      0.79      2194
       I-gpe       0.88      0.38      0.53        60
       I-nat       0.00      0.00      0.00        17
       I-org       0.80      0.75      0.78      4917
       I-per       0.86      0.83      0.85      5093
       I-tim       0.85      0.70      0.77      1950
           O       0.99      1.00      1.00    671658

    accuracy              

c:\Users\Lenovo\anaconda3\envs\preprocess\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
from seqeval.metrics import classification_report as seqeval_cs

print(seqeval_cs(Y_true, Y_pred, digits =4))

c:\Users\Lenovo\anaconda3\envs\preprocess\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         art     0.0000    0.0000    0.0000       140
         eve     0.8500    0.1504    0.2556       113
         geo     0.8296    0.8809    0.8545     11090
         gpe     0.9533    0.9352    0.9442      4755
         nat     1.0000    0.0645    0.1212        62
         org     0.6874    0.6258    0.6552      5952
         per     0.7258    0.6931    0.7091      5053
         tim     0.8661    0.8225    0.8437      6118

   micro avg     0.8148    0.7961    0.8053     33283
   macro avg     0.7390    0.5216    0.5479     33283
weighted avg     0.8097    0.7961    0.8006     33283



In [13]:
# accuracy
from seqeval.metrics import accuracy_score

accuracy = round(accuracy_score(Y_true, Y_pred)*100,2)
print('Accuracy test = {}%'.format(accuracy))

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(YT, YP)
labels=["B-art", "B-eve", "B-geo", "B-gpe", "B-nat", "B-org", "B-per", "B-tim",
        "I-art", "I-eve", "I-geo", "I-gpe", "I-nat", "I-org", "I-per", "I-tim", "O"]
acc_class = cm.diagonal()/cm.sum(axis=1)
df_acc_class = pd.DataFrame({'Label': labels,
                             'Accuracy': acc_class})
df_acc_class

Accuracy test = 98.54%


,Label,Accuracy
0,B-art,0.000000
1,B-eve,0.017699
2,B-geo,0.892606
3,B-gpe,0.936698
4,B-nat,0.064516
5,B-org,0.664987
6,B-per,0.783056
7,B-tim,0.858450
8,I-art,0.000000
9,I-eve,0.029703


### Lưu lại mô hình và bộ trọng số đã train.

In [14]:
model.save_weights('NER_C1_LSTM_weights_22521117.weights.h5')
model.save('NER_C1_LSTM_22521117.h5')